In [1]:
import pandas as pd
import ebcdic
import codecs

file = r'C:\PublicData\Texas\TXRRC\index\dbf900.ebc' ##Local storage location
##file origin: ftp://ftpe.rrc.texas.gov/shfwba/dbf900.ebc.gz

with open(file, 'rb') as ebcdic:
    data = ebcdic.read()

ascii_txt = codecs.decode(data, 'cp1140')

In [2]:
split_records = []
n  = 247 ##Unknown if this holds true for all versions of this file or for other files on TXRRC

for index in range(0, len(ascii_txt), n):
    split_records.append(ascii_txt[index : index + n])

#### Checking different fields and the formats
The lat and long data is a COBOL comp3 format decimal PIC S9(3)V9(7) which should translate to DDD.DDDDDDD

Getting values in the last position that are letters or "{" that do not make sense. From;
http://3480-3590-data-conversion.com/article-packed-fields.html
the last position should indicate the sign of the number, but there isn't a consistent decoding to make that work out.

The TXRRC map shows a 7decimal place percision when exported, but only 6 when reviewed though the info tool, these might be coming from different file sources. The 6decimal percision can be grabbed from the current field without the last letter code

In [4]:
API = None
ct = 0

sample_records = split_records[4000000:5200000]

for record in sample_records:
    if record.startswith('01'):
        API = '42'+record[2:10]
    if record.startswith('13'):
        #print(record)
        #print(ct,'API:',API,'lat:',record[132:142],'long:',record[142:152])
        try:
            #Checking to see if value for WB-PLANE-COORDINATE-EAST can be converted to number
            #This is for testing to check the coordinate reference system in file
            val = int(record[159:168])
        except:
            val = 0
            
        if val > 0:
            
            #print(ct,'API:',API,'check:',record[152:])
            
            #WGS84 lat-long
            lat = float(str(int(record[132:135]))+'.'+str(int(record[135:141])))
            long = float('-'+str(int(record[142:145]))+'.'+str(int(record[145:151])))
            
            ##Unknown what coordinate reference system these are in
            coordZone = record[157:159]
            coordE = float(str(int(record[159:167]))+'.'+str(int(record[167])))
            coordN = float(str(int(record[169:177]))+'.'+str(int(record[177])))
            
            print(API + ", "+ str(lat) + ", "+ str(long) + ", "+ str(coordZone)+", "+str(coordE) + ", "+ str(coordN))
            
        ct+=1

4210334286, 31.645677, -102.36856, 03, 1366819.0, 725497.0
4210334312, 31.639793, -102.363698, 03, 1369298.0, 723425.0
4210334323, 31.61775, -102.325234, 03, 1379950.0, 715040.0
4210334324, 31.622356, -102.325796, 03, 1379975.0, 715770.0
4210334325, 31.624079, -102.321936, 03, 1381190.0, 717363.0
4210334326, 31.615237, -102.324366, 03, 1380200.0, 714120.0
4210334405, 31.582741, -102.346673, 03, 1373169.0, 702567.0
4210334411, 31.589944, -102.356245, 03, 1370282.0, 705157.0
4210334431, 31.641623, -102.390659, 03, 1359916.0, 724149.0
4210334432, 31.629762, -102.395107, 03, 1358452.0, 719861.0
4210334446, 31.634446, -102.38946, 03, 1360225.0, 721587.0
4210334447, 31.622224, -102.389147, 03, 1360256.0, 717085.0
4210334449, 31.553238, -102.325206, 03, 379706.0, 691633.0
4210334483, 31.638426, -102.385239, 03, 1361581.0, 722955.0
4210334484, 31.573544, -102.340905, 03, 1374950.0, 699106.0
4210334565, 31.275874, -102.363625, 03, 1365900.0, 590980.0
4210500780, 30.704364, -101.172894, 03, 1736

4211532823, 32.770492, -102.14929, 02, 571561.7, 433123.5
4211532852, 32.613821, -101.719338, 02, 701302.0, 370622.0
4211532860, 32.675517, -101.79793, 02, 678019.9, 394098.3
4211532862, 32.676391, -101.735326, 02, 697328.5, 393539.8
4211532879, 32.637287, -101.736876, 02, 696250.2, 379365.0
4211532895, 32.673611, -101.737287, 02, 696487.3, 392681.3
4211532896, 32.694488, -101.739927, 02, 696151.5, 400195.8
4211532931, 32.877795, -102.163152, 02, 569040.0, 472334.0
4211532933, 32.927499, -102.179828, 02, 564731.4, 490625.9
4211532959, 32.6413, -101.736308, 02, 696483.8, 380816.6
4212331812, 29.154456, -97.48093, 04, 2623044.0, 485455.0
4212331831, 29.31742, -97.120722, 04, 2600584.0, 440463.0
4212530591, 33.418853, -101.28502, 02, 923704.7, 655493.0
4212530592, 33.495037, -100.647721, 02, 1040657.0, 679510.0
4212733070, 28.566948, -99.719567, 04, 1769128.6, 267357.5


In [18]:
len(split_records)

25198781